In [1]:
import pandas as pd
from grape import Graph
from grape.edge_prediction import PerceptronEdgePrediction
from grape.embedders import FirstOrderLINEEnsmallen

2023-06-18 15:01:17.975910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
graph = Graph.from_csv(
    directed=False,
    node_path="./data/node_types_and_names.csv",
    node_list_header=True,
    nodes_column_number=1,
    node_list_node_types_column_number=3,
    node_ids_column_number=2,
    edge_path="./data/edges_list.csv",
    edge_list_header=True,
    sources_column_number=1,
    destinations_column_number=2,
    weights_column_number=3,
    edge_list_numeric_node_ids=True,
    load_node_list_in_parallel=False,
    load_edge_list_in_parallel=False,
    verbose=True,
    name="Lotus"
)

In [3]:
from grape.datasets import get_okapi_tfidf_weighted_textual_embedding

In [4]:
index = pd.read_csv("./data/molecule_SMILES.csv", index_col=0).iloc[:, 0]

In [6]:
embedding_mol = pd.DataFrame(get_okapi_tfidf_weighted_textual_embedding(
    "./data/molecule_SMILES.csv",
    separator=",",
    header=True,
    # The k1 and b are BM25 parameters.
    # They may be tuned using a number of hyper-parameter optimization techniques.
    k1 = 1.5,
    b = 0.75,
    columns=["structure_smiles_2D"],
    # Just set here any BERT-based model from huggingface.
    pretrained_model_name_or_path="seyonec/ChemBERTa-zinc-base-v1"
),index=index)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 were not used when initializing BertModel: ['roberta.encoder.layer.1.attention.output.dense.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.dense.weight', 'roberta.encoder.layer.5.attention.self.value.weight', 'roberta.encoder.layer.4.attention.output.LayerNorm.weight', 'roberta.encoder.layer.5.attention.output.dense.weight', 'roberta.encoder.layer.1.attention.self.value.bias', 'roberta.encoder.layer.4.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.self.query.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.self.key.bias', 'roberta.encoder.layer.1.output.dense.weight', 'rober

In [7]:
embedding_species = pd.read_csv("./data/species_features_dummy.csv.gz", index_col=0)

In [8]:
%%time
from grape.embedders import DeepWalkSkipGramEnsmallen

# In this example we are running a lighter parametrization,
# but you can and should explore more complex ones.
deepwalk_embedding = DeepWalkSkipGramEnsmallen(
    embedding_size=100,
    iterations=1,
    epochs=3
).fit_transform(
    graph,
)

CPU times: user 2min 46s, sys: 563 ms, total: 2min 47s
Wall time: 45 s


In [9]:
deepwalk_embedding=deepwalk_embedding.get_node_embedding_from_index(0)

In [11]:
from grape.edge_prediction import KipfGCNEdgePrediction

model = KipfGCNEdgePrediction(
    # We run only 20 epochs in this example.
    # Likely you will want to run more!
    epochs=3,
    # We use two GCN layers
    number_of_graph_convolution_layers=3,
    # And two dense layers on top
    number_of_ffnn_head_layers=2,
    # We allow the model to use traditional edge metrics
    # such as Adamic-Adar and Jaccard
    use_edge_metrics=True,
    # We make also a node embedding layer, so the GCN
    # can learn its own node embedding.
    use_node_embedding=True,
    # We show the loading bars while training
    verbose=True
)

# And we train the model!
model.fit(
    graph=graph,
    node_features=[embedding_mol, embedding_species, deepwalk_embedding]
)


ValueError: The provided node features have 48599 rows but the provided graph Lotus has 71410 nodes. Maybe these features refer to another version of the graph or another graph entirely?

In [58]:
from grape.e

Help on package edge_prediction:

NAME
    edge_prediction - Submodule providing models for edge prediction.

PACKAGE CONTENTS
    edge_prediction_catboost (package)
    edge_prediction_ensmallen (package)
    edge_prediction_evaluation
    edge_prediction_lightgbm (package)
    edge_prediction_model
    edge_prediction_sklearn (package)
    edge_prediction_tensorflow (package)
    edge_prediction_xgboost (package)
    sklearn_like_edge_prediction_adapter

CLASSES
    embiggen.edge_prediction.edge_prediction_model.AbstractEdgePredictionModel(embiggen.utils.abstract_models.abstract_classifier_model.AbstractClassifierModel)
        embiggen.edge_prediction.edge_prediction_ensmallen.perceptron.PerceptronEdgePrediction
        embiggen.edge_prediction.edge_prediction_tensorflow.gcn.GCNEdgePrediction(embiggen.utils.abstract_edge_gcn.AbstractEdgeGCN, embiggen.edge_prediction.edge_prediction_model.AbstractEdgePredictionModel)
            embiggen.edge_prediction.edge_prediction_tensorflow.gnn